In [18]:
# === Cellule 1 : imports & chemins ===
import os, re, gc
import pandas as pd
import numpy as np

# Pour l’affichage
pd.set_option("display.max_columns", 80)

# Dossiers (adapte si besoin)
DIR_OCC = "Occitanie"
DIR_NAQ = "Nouvelle-Aquitaine"
PATH_LOYERS = "Loyer/loyers_filtre_occitanie_nouvelle_aquitaine.csv"

OUT_DIR_CLEAN = "outputs/clean"
os.makedirs(OUT_DIR_CLEAN, exist_ok=True)

# Colonnes DVF utiles pour l'analyse ventes/loyers (on réduit au strict nécessaire)
DVF_COLS = [
    "idmutation", "datemut", "anneemut", "moismut",
    "coddep", "l_codinsee",
    "valeurfonc", "sbati",
    "libtypbien", "vefa"
]

# Types compacts (gain mémoire)
DVF_DTYPES = {
    "anneemut": "int16",
    "moismut": "int8",
    "coddep": "category",
    "valeurfonc": "float32",
    "sbati": "float32",
    "libtypbien": "category",
    "vefa": "category"
}

# Regex pour extraire tous les codes INSEE (5 chiffres) présents dans l_codinsee
RE_INSEE = re.compile(r"\b\d{5}\b")


In [19]:
# === Cellule 2 : lecteur robuste d'un CSV DVF ===
def read_dvf_csv(path):
    """Lit un CSV DVF (séparateur ;) avec encodage robuste."""
    try:
        df = pd.read_csv(path, sep=";", encoding="utf-8", low_memory=False)
    except UnicodeDecodeError:
        df = pd.read_csv(path, sep=";", encoding="latin1", low_memory=False)
    return df

def list_csvs(folder):
    return sorted([os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".csv")])


In [20]:
# === Cellule 3 : nettoyage & features ===
def clean_one_department(csv_path):
    raw = read_dvf_csv(csv_path)

    # Colonnes utiles
    cols = [c for c in DVF_COLS if c in raw.columns]
    df = raw[cols].copy()

    # Dates et types
    if "datemut" in df.columns:
        df["datemut"] = pd.to_datetime(df["datemut"], errors="coerce")
    for c, t in DVF_DTYPES.items():
        if c in df.columns:
            try:
                df[c] = df[c].astype(t)
            except Exception:
                pass

    # --- Filtrage ---
    if "libtypbien" in df.columns:
        df = df[df["libtypbien"].str.contains("MAISON|APPARTEMENT", case=False, na=False)]
    if "valeurfonc" in df.columns:
        df = df[df["valeurfonc"] > 50000]
    if "sbati" in df.columns:
        df = df[(df["sbati"].notna()) & (df["sbati"] > 20)]

    # Prix/m²
    df["prix_m2"] = (df["valeurfonc"] / df["sbati"]).astype("float32")

    # Extraire codes INSEE
    insee_lists = df["l_codinsee"].astype(str).apply(lambda s: RE_INSEE.findall(s))
    df = df.assign(INSEE_C=insee_lists).explode("INSEE_C").dropna(subset=["INSEE_C"])

    # Colonne département
    if "coddep" in df.columns:
        df["DEP"] = df["coddep"].astype(str)
    else:
        dep_guess = re.findall(r"d(\d{2,3})", os.path.basename(csv_path))
        df["DEP"] = dep_guess[0] if dep_guess else "00"

    # Colonnes finales utiles
    keep = [
        "idmutation",  # ⚡ corrigé ici
        "INSEE_C", "DEP", "anneemut", "moismut",
        "prix_m2", "valeurfonc", "sbati",
        "libtypbien", "vefa"
    ]
    keep = [c for c in keep if c in df.columns]
    df = df[keep].reset_index(drop=True)

    # Compactage
    for c in ["INSEE_C", "DEP", "libtypbien", "vefa"]:
        if c in df.columns:
            df[c] = df[c].astype("category")

    return df


In [21]:
# === Cellule 4 : loyers ===
def load_loyers(path_csv):
    # Fichier séparé par ';' avec virgule décimale -> convertir en float
    try:
        L = pd.read_csv(path_csv, sep=";", encoding="utf-8")
    except UnicodeDecodeError:
        L = pd.read_csv(path_csv, sep=";", encoding="latin1")

    # Normalisation colonnes attendues
    must_have = ["INSEE_C", "DEP", "REG", "LIBGEO", "loypredm2"]
    for c in must_have:
        if c not in L.columns:
            raise ValueError(f"Colonne manquante dans loyers: {c}")

    # Convertir loyers (virgule -> point si string)
    if L["loypredm2"].dtype == "object":
        L["loypredm2"] = (
            L["loypredm2"]
            .str.replace(",", ".", regex=False)
            .str.replace(" ", "", regex=False)
            .astype(float)
        )
    L["loypredm2"] = L["loypredm2"].astype("float32")

    # Colonnes compactes
    L["INSEE_C"] = L["INSEE_C"].astype(str)
    L["DEP"] = L["DEP"].astype(str)
    L["REG"] = L["REG"].astype("int16")
    L["LIBGEO"] = L["LIBGEO"].astype("category")

    return L[["INSEE_C", "DEP", "REG", "LIBGEO", "loypredm2"]]

loyers = load_loyers(PATH_LOYERS)
loyers.head()


,INSEE_C,DEP,REG,LIBGEO,loypredm2
0,64394,64,75,Monpezat,8.555667
1,32450,32,76,Tourdun,8.555667
2,64369,64,75,Maspie-Lalonquère-Juillacq,8.555667
3,64552,64,75,Vialer,8.555667
4,65174,65,76,Estirac,8.555667


In [24]:
# === Cellule 5 : traitement séquentiel par département (version CSV only) ===
def process_region(folder, loyers_df, out_dir=OUT_DIR_CLEAN):
    csvs = list_csvs(folder)
    print(f"➡️  {folder} : {len(csvs)} fichiers trouvés")

    resumes = []

    for i, csv_path in enumerate(csvs, 1):
        dep_code = re.findall(r"d(\d{2,3})", os.path.basename(csv_path))
        dep_code = dep_code[0] if dep_code else f"{i:02d}"

        print(f"\n[{i}/{len(csvs)}] Traitement {os.path.basename(csv_path)} (DEP={dep_code})...")

        # Nettoyage DVF
        df_dep = clean_one_department(csv_path)

        # Sauvegarde DVF propre
        out_clean = os.path.join(out_dir, f"dvf_dep_{dep_code}.csv")
        df_dep.to_csv(out_clean, index=False)
        print(f"✅ Sauvegardé DVF propre : {out_clean}")

        # Fusion DVF + Loyers
        merged = df_dep.merge(loyers_df, on=["INSEE_C", "DEP"], how="inner")

        # Rentabilité brute
        merged["rentabilite_m2"] = (
            merged["loypredm2"] * 12 * merged["sbati"] / merged["valeurfonc"] * 100
        ).astype("float32")

        # Sauvegarde fusion
        out_fus = os.path.join(out_dir, f"fusion_dep_{dep_code}.csv")
        merged.to_csv(out_fus, index=False)
        print(f"✅ Sauvegardé fusion : {out_fus}")

        # Agrégation communale
        agg = merged.groupby(["DEP", "INSEE_C"], as_index=False).agg(
            nb_ventes=("idmutation", "nunique"),        # ⚡ corrigé ici
            prix_m2_moy=("prix_m2", "mean"),
            loyer_m2_moy=("loypredm2", "mean"),
            rentabilite_m2_moy=("rentabilite_m2", "mean")
        )
        out_comm = os.path.join(out_dir, f"communes_dep_{dep_code}.csv")
        agg.to_csv(out_comm, index=False)
        print(f"✅ Sauvegardé agrégation communale : {out_comm}")

        # Résumé départemental
        resumes.append({
            "DEP": dep_code,
            "rows_dvf_clean": len(df_dep),
            "rows_merge": len(merged),
            "communes_couvertes": agg["INSEE_C"].nunique(),
            "prix_m2_moy": float(merged["prix_m2"].mean()) if len(merged) else np.nan,
            "loyer_m2_moy": float(merged["loypredm2"].mean()) if len(merged) else np.nan,
            "rentabilite_m2_moy": float(merged["rentabilite_m2"].mean()) if len(merged) else np.nan,
        })

        del df_dep, merged, agg
        gc.collect()

    return pd.DataFrame(resumes)
resume_occ = process_region(DIR_OCC, loyers)
resume_naq = process_region(DIR_NAQ, loyers)

resume_all = pd.concat([
    resume_occ.assign(REGION="Occitanie"),
    resume_naq.assign(REGION="Nouvelle-Aquitaine")
], ignore_index=True)

resume_all



➡️  Occitanie : 13 fichiers trouvés

[1/13] Traitement mutations_d09.csv (DEP=09)...
✅ Sauvegardé DVF propre : outputs/clean\dvf_dep_09.csv
✅ Sauvegardé fusion : outputs/clean\fusion_dep_09.csv
✅ Sauvegardé agrégation communale : outputs/clean\communes_dep_09.csv

[2/13] Traitement mutations_d11.csv (DEP=11)...
✅ Sauvegardé DVF propre : outputs/clean\dvf_dep_11.csv
✅ Sauvegardé fusion : outputs/clean\fusion_dep_11.csv
✅ Sauvegardé agrégation communale : outputs/clean\communes_dep_11.csv

[3/13] Traitement mutations_d12.csv (DEP=12)...
✅ Sauvegardé DVF propre : outputs/clean\dvf_dep_12.csv
✅ Sauvegardé fusion : outputs/clean\fusion_dep_12.csv
✅ Sauvegardé agrégation communale : outputs/clean\communes_dep_12.csv

[4/13] Traitement mutations_d30.csv (DEP=30)...
✅ Sauvegardé DVF propre : outputs/clean\dvf_dep_30.csv
✅ Sauvegardé fusion : outputs/clean\fusion_dep_30.csv
✅ Sauvegardé agrégation communale : outputs/clean\communes_dep_30.csv

[5/13] Traitement mutations_d31.csv (DEP=31)...
✅ S

,DEP,rows_dvf_clean,rows_merge,communes_couvertes,prix_m2_moy,loyer_m2_moy,rentabilite_m2_moy,REGION
0,09,21232,0,0,NaN,NaN,NaN,Occitanie
1,11,66476,265816,433,1971.802002,9.815851,7.657597,Occitanie
2,12,32206,128524,285,1577.096802,8.558046,7.991147,Occitanie
3,30,103679,414576,349,2395.882568,11.410075,7.145183,Occitanie
4,31,218923,875616,586,2809.688965,12.264627,6.208169,Occitanie
5,32,24391,97520,460,1529.970825,8.800980,8.606750,Occitanie
6,34,215764,862916,342,2959.651123,13.018087,6.497240,Occitanie
7,46,22725,90224,313,1556.439087,8.749647,8.158183,Occitanie
8,48,6994,27524,152,1516.156372,8.252924,8.224405,Occitanie
9,65,33774,135072,466,1697.624268,9.218750,8.189573,Occitanie


In [25]:
# === Cellule 6 : concaténer toutes les communes_dep en un seul fichier ===
def concat_communes(out_dir=OUT_DIR_CLEAN):
    # Lister uniquement les fichiers communes_dep_XX.csv
    files = [
        os.path.join(out_dir, f)
        for f in os.listdir(out_dir)
        if f.startswith("communes_dep_") and f.endswith(".csv")
    ]

    if not files:
        print("⚠️ Aucun fichier communes_dep trouvé dans", out_dir)
        return pd.DataFrame()

    # Charger et concaténer
    dfs = [pd.read_csv(fp) for fp in files if os.path.getsize(fp) > 0]
    allc = pd.concat([df for df in dfs if not df.empty], ignore_index=True)

    # Sauvegarde globale
    out_file = os.path.join(out_dir, "communes_all.csv")
    allc.to_csv(out_file, index=False)
    print(f"✅ Fichier global sauvegardé : {out_file} ({len(allc)} lignes)")

    return allc

# ⚡ Lancer la fusion des communes Occitanie + Nouvelle-Aquitaine
communes_all = concat_communes()
communes_all.head()


✅ Fichier global sauvegardé : outputs/clean\communes_all.csv (8426 lignes)


,DEP,INSEE_C,nb_ventes,prix_m2_moy,loyer_m2_moy,rentabilite_m2_moy
0,11,11001,63,1173.2318,9.236553,11.571651
1,11,11002,33,1582.4867,9.532426,8.664107
2,11,11003,29,1021.2512,8.474989,12.161631
3,11,11004,43,1644.3351,7.992160,7.557476
4,11,11005,134,1712.3025,8.824945,6.903784


In [27]:
# === Cellule 7 : contrôle qualité sur communes_all ===

# Charger communes_all (au cas où on relance séparément)
communes_all = pd.read_csv(os.path.join(OUT_DIR_CLEAN, "communes_all.csv"))

print("✅ communes_all chargé :", communes_all.shape[0], "lignes et", communes_all.shape[1], "colonnes")

# Aperçu des colonnes
print("\nColonnes :", list(communes_all.columns))

# Vérifier quelques communes au hasard
print("\nAperçu de 5 communes aléatoires :")
display(communes_all.sample(5, random_state=42))

# Vérifier s'il y a des valeurs manquantes
print("\nValeurs manquantes par colonne :")
print(communes_all.isna().sum())

# Statistiques descriptives
print("\nStatistiques descriptives principales :")
display(communes_all.describe(include="all"))

# Top 10 communes par rentabilité moyenne
print("\nTop 10 communes par rentabilité_m2_moy :")
top_communes = communes_all.sort_values("rentabilite_m2_moy", ascending=False).head(10)
display(top_communes)

# Bottom 10 communes par rentabilité moyenne
print("\nBottom 10 communes par rentabilité_m2_moy :")
bottom_communes = communes_all.sort_values("rentabilite_m2_moy", ascending=True).head(10)
display(bottom_communes)

# Distribution du nombre de ventes
print("\nDistribution du nombre de ventes par commune :")
display(communes_all["nb_ventes"].describe())


✅ communes_all chargé : 8426 lignes et 6 colonnes

Colonnes : ['DEP', 'INSEE_C', 'nb_ventes', 'prix_m2_moy', 'loyer_m2_moy', 'rentabilite_m2_moy']

Aperçu de 5 communes aléatoires :


,DEP,INSEE_C,nb_ventes,prix_m2_moy,loyer_m2_moy,rentabilite_m2_moy
2835,30,30263,415,1753.5529,9.362774,8.376483
1650,19,19109,24,1614.4503,7.803344,7.009998
1952,23,23136,26,952.5798,7.300178,12.285346
3346,31,31424,2333,2766.7230,11.829565,5.599863
4222,33,33254,46,1516.0674,9.253540,8.820691



Valeurs manquantes par colonne :
DEP                   0
INSEE_C               0
nb_ventes             0
prix_m2_moy           0
loyer_m2_moy          0
rentabilite_m2_moy    0
dtype: int64

Statistiques descriptives principales :


,DEP,INSEE_C,nb_ventes,prix_m2_moy,loyer_m2_moy,rentabilite_m2_moy
count,8426.000000,8426.000000,8426.000000,8426.000000,8426.000000,8426.000000
mean,41.426062,41633.376573,215.020769,1637.146770,9.035386,8.579450
std,23.237370,23218.606506,1528.572939,654.640730,1.337698,1.784497
min,11.000000,11001.000000,1.000000,548.076900,6.407329,0.894154
25%,24.000000,24034.250000,22.000000,1282.483925,8.139324,7.373265
50%,33.000000,33244.500000,48.000000,1509.291850,8.775901,8.506639
75%,64.000000,64366.750000,119.000000,1818.956050,9.688368,9.703257
max,87.000000,87206.000000,93807.000000,28087.926000,17.313202,19.662493



Top 10 communes par rentabilité_m2_moy :


,DEP,INSEE_C,nb_ventes,prix_m2_moy,loyer_m2_moy,rentabilite_m2_moy
1190,17,17116,12,1062.86460,9.255109,19.662493
6520,65,65013,1,548.07690,8.333152,18.245216
6875,65,65381,3,770.63200,7.587695,18.046183
157,11,11160,13,1375.55740,9.300686,17.598822
1708,19,19168,4,591.62370,7.485338,17.316723
226,11,11230,5,752.65125,9.294004,16.778889
6480,64,64531,7,927.06866,9.935357,16.614138
1880,23,23059,2,577.50000,6.849565,16.585426
6362,64,64412,10,866.18567,8.959445,16.558410
1126,17,17049,12,885.14410,9.517236,16.536062



Bottom 10 communes par rentabilité_m2_moy :


,DEP,INSEE_C,nb_ventes,prix_m2_moy,loyer_m2_moy,rentabilite_m2_moy
5087,40,40242,1,11992.6200,8.936037,0.894154
1352,17,17286,402,7271.0996,12.529368,2.646175
4204,33,33236,2647,9978.2270,13.211596,2.734483
1128,17,17051,716,6686.7840,12.529367,2.861798
1433,17,17369,621,6158.1350,12.529368,2.883278
1363,17,17297,538,5752.8910,12.529368,3.165480
1233,17,17161,783,6068.2700,12.529368,3.175084
1195,17,17121,482,5884.2617,12.529368,3.223100
242,11,11247,2,3420.2646,8.298773,3.255263
1425,17,17360,853,5509.1960,12.529367,3.264702



Distribution du nombre de ventes par commune :


count     8426.000000
mean       215.020769
std       1528.572939
min          1.000000
25%         22.000000
50%         48.000000
75%        119.000000
max      93807.000000
Name: nb_ventes, dtype: float64